In [ ]:
import pandas as pd
from py2neo import authenticate, Graph, Node, Relationship

# Log-in to Neo4j server.
authenticate('localhost:7474', 'neo4j', 'admin')

# Create lineage graph.
graph = Graph()
graph.delete_all()

# TODO: Use a combo of dataset name and hash of contents as unique constraint?
graph.schema.drop_uniqueness_constraint('Dataset', graph.schema.get_uniqueness_constraints('Dataset'))
graph.schema.drop_uniqueness_constraint('Job', graph.schema.get_uniqueness_constraints('Job'))

graph.schema.create_uniqueness_constraint('Dataset', 'name')
graph.schema.create_uniqueness_constraint('Job', 'uid')

dataset1 = graph.merge_one('Dataset', 'name', 'test1.csv')
dataset2 = graph.merge_one('Dataset', 'name', 'test2.csv')

job1 = graph.merge_one('Job', 'uid', 'cp-3792')
job1['name'] = 'cp'
job1['pid'] = 3792
graph.push(job1)

graph.create_unique(Relationship(dataset1, 'IS_READ_BY', job1, timestamp=1))
graph.create_unique(Relationship(job1, 'WRITES_TO', dataset2, timestamp=2))

query = """
    MATCH (n)
    WITH n, rand() AS random
    ORDER BY random
    OPTIONAL MATCH (n)-[r]->(m)
    RETURN n AS source_node,
           id(n) AS source_id,
           r,
           m AS target_node,
           id(m) AS target_id
    """

results = graph.cypher.execute(query)
for row in results:
    print row

query = """
    MATCH (job:Job) RETURN job.name AS job, job.pid as PID
"""

results = graph.cypher.execute(query)
for row in results:
    print row